# Choice modelling


Today we will look at the example of Choice modelling task.

The practice is based on tutorials: https://www.linkedin.com/pulse/marketing-analytics-series-part-1-customer-choice-nguyen-u-s-cma/ and https://www.linkedin.com/pulse/conjoint-analysis-simple-python-implementation-prajwal-sreenivas/ 
(you need to use VPN in order to access Linkedin site)


**The Customer Choice (Logit) model** is an individual-level model helping to understand factors behind each person's purchase decisions. Through quantitative analysis, companies and marketing executives can gain insight on how features such as prices, design, and durability influence the customer's choice of a brand. Customer Choice model is therefore widely used in various marketing areas, including, but not limited to, customer segmentation, product design, pricing or customer churn management.

The inputs for Customer Choice Models are usually:

+ Customer's choice data for alternative offerings, each containing specific attributes.
+ Customer's ratings of different alternatives.

![](choice-ex.png)

**Choice modelling task in simple words:**
    
we observe that the customer makes a choice among several options (for example, if a customer comes into the store and purchases some particular chocolate, he or she chooses thi chocolate from many others). Each of the chocolates has its own characteristics - price, brand, favor etc. And customer has some characteristics as well and we may have information about his/her other choices. 

So, we have choices of chocolates for a set of customers. Now we can use some model that will help us to understand the relationship between the attributes of products and customers’ choices among sets of products.

Again, target variable is choice of some product. There are some finite number of products. If there are only two of them, we can consider this task as a binary classification. If there are more than two products, then we can use algorithms for multi-class classification. We also need interpretable models, so that we can interpret the results after its training and understand the relationships between the features and the target variable (choice).

So, it seems that multinomial logit model will be a good choice.

## Data

Here is some list of choice data I found: https://github.com/alvarogutyerrez/TheDiscreteChoiceDataBank.
        
We will use data on **school transportation choice by students in Dresden**. 

**The link to the data**: https://github.com/svenne0815/DresdenModeChoiceData/blob/master/DDModeChoice.txt.


Article where this data is used: https://www.sciencedirect.com/science/article/abs/pii/S0966692320309492?dgcid=coauthor.


In [95]:
data = pd.read_csv('transport_choice_data.txt', sep='\t')

In [79]:
data.head()

,ID,Choice,Distance,School_location,Grade,Age,Gender,CarAvail,Season,CB_location,Leistung
0,1,2,6.03139,0,9,15,0,0,0,0,152.98
1,2,2,3.48652,0,9,15,1,0,0,1,64.67
2,3,4,5.33100,0,9,14,1,1,0,0,147.19
3,4,3,3.75005,0,9,15,0,0,0,0,1287.90
4,5,2,2.04100,0,9,15,1,0,0,1,35.36


Here is the variables descriptions from the article

![](variables.png)

In [80]:
data.groupby(['Season', "Choice"]).count()['ID']/data.shape[0]*100

Season  Choice
0       1          9.887798
        2         13.242169
        3         24.625993
        4          2.244039
1       1         11.827957
        2          4.102384
        3         30.014025
        4          4.055633
Name: ID, dtype: float64

Let's name categories:

+ 1 - Walk
+ 2 - Bike
+ 3 - Public transport
+ 4 - Car

(this is only my guess, as you see the proportions of each transport are different from those presented in the article, and there is no any codings for data in the data itself, so let's assume that mt guess is right)

In [88]:
# selecting columns we need
data = data[['Choice', 'Distance', 'Grade', 'Age', 'Gender', 'CarAvail', 'Season']]

Other categorial columns codings:
    
+ Season (1-Winter, 0-Summer)
+ CarAvail (1-available, 0-not available)


In [96]:
data['CarAvail'] = data['CarAvail'].apply(lambda x: 'available' if x == 1 else 'not available')
data['Season'] = data['Season'].apply(lambda x: 'winter' if x == 1 else 'summer')

## Modelling with Multinomial Logit


Practice on mnlogit from the previous year: https://drive.google.com/file/d/1NzGHvmWddAFOMMr3u-vrMLzH1qq050-9/view?usp=sharing

In [97]:
from statsmodels.formula.api import mnlogit

multi_model = mnlogit('Choice ~ C(Gender) + Age + Grade + Distance + C(CarAvail) + C(Season)', 
                      data=data, distr='logit').fit()
multi_model.summary()

Optimization terminated successfully.
         Current function value: 0.532998
         Iterations 10


/home/alena-pestova/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['distr']
  warnings.warn(msg, ValueWarning)
/home/alena-pestova/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['distr']
  warnings.warn(msg, ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 Choice   No. Observations:                 8556
Model:                        MNLogit   Df Residuals:                     8535
Method:                           MLE   Df Model:                           18
Date:                Thu, 06 Oct 2022   Pseudo R-squ.:                  0.5324
Time:                        18:00:27   Log-Likelihood:                -4560.3
converged:                       True   LL-Null:                       -9753.0
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================
                    Choice=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -5.1572      0.440    -11.730      0.000      -6.019      -4.295
C(Gender)[T.1]                  -0.5906      0.101     -5.871      0.000      -0.788      -0.393
C(CarAvail)[T.not available]    -0.3051      0.227     -1.343      0.179      -0.750       0.140
C(Season)[T.winter]             -1.5465      0.105    -14.731      0.000      -1.752      -1.341
Age                              0.1015      0.046      2.226      0.026       0.012       0.191
Grade                            0.0062      0.053      0.117      0.907      -0.098       0.110
Distance                         3.4868      0.116     30.155      0.000       3.260       3.713
------------------------------------------------------------------------------------------------
                    Choice=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -6.0806      0.466    -13.055      0.000      -6.993      -5.168
C(Gender)[T.1]                   0.1432      0.104      1.381      0.167      -0.060       0.347
C(CarAvail)[T.not available]     0.3955      0.241      1.638      0.101      -0.078       0.869
C(Season)[T.winter]              0.1210      0.106      1.143      0.253      -0.087       0.329
Age                             -0.0050      0.050     -0.099      0.921      -0.103       0.094
Grade                           -0.0765      0.057     -1.332      0.183      -0.189       0.036
Distance                         4.3361      0.118     36.875      0.000       4.106       4.567
------------------------------------------------------------------------------------------------
                    Choice=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -4.6467      0.530     -8.768      0.000      -5.685      -3.608
C(Gender)[T.1]                   0.1856      0.164      1.133      0.257      -0.136       0.507
C(CarAvail)[T.not available]    -4.9300      0.242    -20.375      0.000      -5.404      -4.456
C(Season)[T.winter]              0.5912      0.169      3.501      0.000       0.260       0.922
Age                              0.0234      0.053      0.438      0.661      -0.081       0.128
Grade                           -0.1271      0.069     -1.847      0.065      -0.262       0.008
Distance                         4.2776      0.120     35.544      0.000       4.042       4.514
================================================================================================
"""

## Interpeting the results

(reminder from the practice on mnlogit)

### Extra comments:

the very same interpreataion as it was for binary logreg with the only difference that we have
- one reference category (here we have immigration_policy=1) 
- then we have 3 other categories of the dependent variable

in the binary logreg we have one reference and another one and we look at the log-odds

$$\log(\frac{P(Y==1)}{P(Y==0)}) = \beta_0 + \beta_1 * x_1 + ... + \beta_n * x_n$$


here we have several similar formulas with the only difference in nominators (and obviously in coefficients)

$$\log(\frac{P(Y=i)}{P(Y=0)}) = \beta_{i0} + \beta_{i1} * x_1 + ... + \beta_{in} * x_n$$

For ex, in our case:

$$\log(\frac{P(immigration\_policy==2)}{P(immigration\_policy==1)}) = 1.2362 + 0.1623 * Sex +  0.0010 * Age$$

$$\log(\frac{P(immigration\_policy==3)}{P(immigration\_policy==1)}) = 2.0210 + -0.0902 * Sex +  0.0070 * Age$$


More reminders: 

*Interpreting coefficients*

+ β (log-odds) refers to whether the mean of y increases or decreases as x increases. So, it shows whether there is a positive or negative relationship between the target variable and the feature.

+ odds coefficient A shows you that "when the feature increases by 1 unit the target variable increases by a factor of A" (the target variable will be multiplied by A)

+ Average marginal effects: for categorical variables with more than two possible values, the marginal effects show you the difference in the predicted probabilities for cases in one category relative to the reference category.

With binary independent variables, marginal effects measure discrete change, i.e. how the predicted probabilities change as the binary independent variable changes from 0 to 1.
Marginal effects for continuous variables often provide a good approximation to the amount of change in Y that will be produced by a 1-unit change in X.

    

This is how we can calculate odds (from log-odds)

In [98]:
odds = np.exp(multi_model.params)

In [99]:
odds.columns = ['Bike', 'Transport', 'Car']

In [100]:
odds

,Bike,Transport,Car
Intercept,0.005758,0.002287,0.009593
C(Gender)[T.1],0.553970,1.154012,1.203978
C(CarAvail)[T.not available],0.737032,1.485089,0.007227
C(Season)[T.winter],0.212991,1.128634,1.806225
Age,1.106786,0.995059,1.023631
Grade,1.006248,0.926349,0.880616
Distance,32.682655,76.409644,72.069268


In [102]:
print(multi_model.get_margeff(at ='overall').summary())

       MNLogit Marginal Effects      
Dep. Variable:                 Choice
Method:                          dydx
At:                           overall
                    Choice=1      dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
C(Gender)[T.1]                   0.0132      0.005      2.638      0.008       0.003       0.023
C(CarAvail)[T.not available]     0.0111      0.011      1.031      0.303      -0.010       0.032
C(Season)[T.winter]              0.0401      0.005      8.146      0.000       0.030       0.050
Age                             -0.0028      0.002     -1.192      0.233      -0.008       0.002
Grade                            0.0018      0.003      0.664      0.507      -0.004       0.007
Distance                        -0.2062      0.001   -154.767      0.000      -0.209      -0.204
------------------------------------------------------------------------

## Predicting

In [112]:
data['Distance'].quantile(0.75)

4.80713

In [119]:
# Predictions for the average and extreme values of predictors 
# tables (change one variable, remain the others constant)
# out variables C(Gender) + Age + Grade + Distance + C(CarAvail) + C(Season)

pd.options.display.float_format = '{:.4f}'.format

pred_table = pd.DataFrame(columns=["Gender", "Age", "Grade", "Distance", "CarAvail", "Season"])
pred_table['Distance'] = data['Distance'].min(), data['Distance'].quantile(0.25), data['Distance'].quantile(0.5), data['Distance'].quantile(0.75), data['Distance'].max()
pred_table['Gender'] = data['Gender'].mode()[0]
pred_table['Age'] = data['Age'].mean()
pred_table['Grade'] = data['Grade'].mean()
pred_table['CarAvail'] = data['CarAvail'].mode()[0]
pred_table['Season'] = data['Season'].mode()[0]

predictions = multi_model.predict(pred_table)
predictions.columns = ['P_Walk', 'P_Bike', 'P_Transport', 'P_Car']

pred_table = pd.concat([pred_table, predictions], axis=1)
pred_table

,Gender,Age,Grade,Distance,CarAvail,Season,P_Walk,P_Bike,P_Transport,P_Car
0,1,14.7108,8.7288,0.0368,not available,summer,0.9854,0.0124,0.0022,0.0000
1,1,14.7108,8.7288,1.3420,not available,summer,0.3533,0.4202,0.2223,0.0043
2,1,14.7108,8.7288,2.7480,not available,summer,0.0022,0.3593,0.6273,0.0111
3,1,14.7108,8.7288,4.8071,not available,summer,0.0000,0.0894,0.8966,0.0141
4,1,14.7108,8.7288,17.8006,not available,summer,0.0000,0.0000,0.9927,0.0073


## Market share

In a customer choice analysis, we may want to analyze how the choice changes with some variable change. It can be some customer characteritics or product attribute.


We can also look at the predicted market share of each transport choice.

NOTE: this is a little bit stupid example, because we predict the choices on the same data that we used for training. Actually, we have real labels here and we now real shares of each transport. In real research, we may want to calculate probabilites and shares for some new data that we did not use for training.

In [121]:
multi_model.predict(data)

,0,1,2,3
0,0.0000,0.0709,0.9163,0.0127
1,0.0001,0.2408,0.7465,0.0126
2,0.0000,0.0295,0.2414,0.7291
3,0.0000,0.3459,0.6438,0.0102
4,0.0357,0.5010,0.4550,0.0083
...,...,...,...,...
8551,0.0000,0.0013,0.9801,0.0186
8552,0.0008,0.1405,0.8365,0.0222
8553,0.0000,0.0047,0.1968,0.7985
8554,0.0017,0.0984,0.8754,0.0244


In [133]:
# predictions of the class
predicted_classes = multi_model.predict(data).apply(lambda x: np.argmax(x), axis=1)

In [134]:
predicted_classes

0       2
1       2
2       3
3       2
4       1
       ..
8551    2
8552    2
8553    3
8554    2
8555    2
Length: 8556, dtype: int64

In [137]:
shares = np.unique(predicted_classes, return_counts=True)[1]

In [140]:
# market shares of each transport
shares / np.sum(shares)

array([0.23772791, 0.11816269, 0.57678822, 0.06732118])

What is more, it may be interested how these market shares change when we change some characteritics of the people and/or product.

# Conjoint Analysis

**What is Conjoint Analysis?**

Conjoint Analysis, short for "consider jointly" is a marketing insight technique that provides consumers with combinations, pairs or groups of products that are a combination of various features and ask them what they prefer. The product is described by a number of attributes and each attribute has several levels.

One of the greatest strengths of Conjoint Analysis is its ability to develop market simulation models that can predict consumer behavior to changes in the product. Conjoint Analysis can be applied to a variety of difficult aspects of the Market research such as product development, competitive positioning, pricing pricing, product line analysis, segmentation and resource allocation.

The difference from choice model  - we have not only the choice of a customer, but also his/her rankings of the products. They come from the experimantal design - the customer chooses from a set of options, and this options usually represent some similar products with some varying characteristics.

The tutorial on conjoint analysis: https://ariepratama.github.io/How-to-do-conjoint-analysis-in-python/.

## Task for you


1. Show how the probabilities of choosing each option of school transportation change when we vary other predictors. Choose 2 predictors, vary each of them with remaining other variables constant, calculate predictions and plot the probabilities. Make conclusions on the relationship between predictors and target variable.

NOTE: this is the same thing that was in Predicting part of this practice. So, you need to choose one predictor, make a table where you vary the values of this predictor (assign constant values to other variables). Then predict probabilites, for each row and plot how they change with different values of chosen predictor. Then choose some other predictor and do the same thing. So, you should analyse the relationship between chosen predictors and target variable separately.

The example of plot we draw in the previous year. Here we vary the age variable and plot the probabilites of some 4 groups for some age.
![](plot1.png)